# Lesson 83

In [1]:
#from sklearn.datasets import load_boston
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np


In [6]:
# Gather Data
#boston_dataset = load_boston()
boston_dataset = fetch_openml(name='boston', version=1)
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)
#features.dtypes
features['CHAS'] = features['CHAS'].astype('float64')
features['RAD'] = features['RAD'].astype('float64')
#features.dtypes

CRIM       float64
ZN         float64
CHAS       float64
NOX        float64
RM         float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
dtype: object

In [ ]:
log_prices = np.log(boston_dataset.target).to_numpy()
# add to_numpy() method to force return a Array otherwise returns a serie
#print(type(log_prices))
target = pd.DataFrame(log_prices, columns=['PRICE'])
#print(target)

        PRICE
0    3.178054
1    3.072693
2    3.546740
3    3.508556
4    3.589059
..        ...
501  3.109061
502  3.025291
503  3.173878
504  3.091042
505  2.476538

[506 rows x 1 columns]


In [ ]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

#property_stats = features.mean().to_numpy().reshape(1,11)
property_stats = features.mean().values.reshape(1, 11)
#property_stats

(1, 11)